Let's try the recordlinkage package for python!

In [ ]:
import pandas as pd
import recordlinkage as rl
import csv
import utils

First read in some appropriate small subset

# Manage training data

In [ ]:
df_train = pd.read_pickle("tmp.pickled")

In [ ]:
relevant = set((1845, 1850, 1860, 1880, 1885))

def buildLinksDict(fd):
    reader = csv.reader(fd, delimiter="|")
    next(reader) # skip header
    links = {year: {} for year in relevant} # year -> (kip,løb) -> linkID
    for row in reader:
        year = utils.extractYear(row[1])
        if year is None:
            continue
        if year not in relevant:
            continue
        kipløb = (row[2], int(row[3]))
        links[year][kipløb] = row[0]
    return links

with (utils.datadir / "links" / "matches.csv").open("r") as fd:
    dm = buildLinksDict(fd)
with (utils.datadir / "links" / "notmatches.csv").open("r") as fd:
    dn = buildLinksDict(fd)

In [ ]:
df_train["LinkID"] = ""

This way is stupid. Indexing by the three things is way cooler. But I don't know how to get "back out" again.

In [ ]:
indices = []
linkIDs = []
for t in df_train.itertuples():
    s = dm[t.FT]
    key = (t.Kipnr, t.Løbenr)
    if key in s:
        indices.append(t.Index)
        linkIDs.append(s[key])
df_train.loc[indices, "LinkID"] = linkIDs

Continue

In [ ]:
df_train = df_train[df_train.LinkID != ""]

In [ ]:
groups = df_train.groupby("LinkID")

In [ ]:
l = []
for group, data in groups:
    l.append(data.index)

In [ ]:
mindex_train = pd.MultiIndex.from_tuples(l, names=["id1", "id2"])

In [ ]:
indexer = rl.SortedNeighbourhoodIndex(on="Fødeår", window=5)

In [ ]:
by_year = df_train.groupby("FT")

In [ ]:
for year, data in by_year:
    print(year, len(data))

In [ ]:
dfA = by_year.get_group(1845)

In [ ]:
dfB = by_year.get_group(1850)

In [ ]:
pairs = indexer.index(dfA, dfB)

In [ ]:
len(pairs)

In [ ]:
comparer = rl.Compare()
comparer.string("Navn", "Navn")
comparer.string("Fødested", "Fødested")
comparer.string("Erhverv", "Erhverv")
comparer.string("Position", "Position")
comparer.string("Civilstand", "Civilstand")
comparer.numeric("Fødeår", "Fødeår", scale=1)

In [ ]:
features_train = comparer.compute(pairs, dfA, dfB)

# Train classifier

In [ ]:
matches = features[features.sum(axis=1) > 4.5]

In [ ]:
for t in matches.itertuples():
    a, b = t[0]
    print(dfA.loc[a])
    print(dfB.loc[b])
    #print(t[0][0])
    #print(t[1])

In [ ]:
lr = rl.LogisticRegressionClassifier()

In [ ]:
svm = rl.SVMClassifier()

In [ ]:
nb = rl.NaiveBayesClassifier()

In [ ]:
svm.learn(features, (features.index & mi))

In [ ]:
nb.learn(features_train, (features_train.index & mindex_train))

In [ ]:
len(lr.predict(features_train))

In [ ]:
len(svm.predict(features_train))

In [ ]:
len(mi)

In [ ]:
len(nb.predict(features_train))

In [ ]:
nb.prob(features_train)

In [ ]:
df_train.loc[[1163538, 1953896]]

# Try the classifier!

In [ ]:
df = pd.read_pickle("tmp.pickled")

In [ ]:
df = df[df.Navn.str.startswith("Da")]

In [ ]:
df = df[df.Køn==False]

In [ ]:
by_year = df.groupby("FT")

In [ ]:
for year, data in by_year:
    print(year, len(data))

In [ ]:
dfA = by_year.get_group(1845)

In [ ]:
dfB = by_year.get_group(1850)

Do some indexing!

In [ ]:
indexer = rl.FullIndex()

In [ ]:
indexer = rl.SortedNeighbourhoodIndex(on="Fødeår", window=3)

In [ ]:
pairs = indexer.index(dfA, dfB)

In [ ]:
len(pairs)

In [ ]:
comparer = rl.Compare()
comparer.string("Navn", "Navn", method="jarowinkler")
comparer.string("Fødested", "Fødested")
comparer.string("Erhverv", "Erhverv")
comparer.string("Position", "Position")
comparer.string("Civilstand", "Civilstand")
comparer.numeric("Fødeår", "Fødeår", scale=1)

In [ ]:
features = comparer.compute(pairs, dfA, dfB)

In [ ]:
lr_res = lr.predict(features)

In [ ]:
svm_res = svm.predict(features)

In [ ]:
svm_res[0]

In [ ]:
import pickle

In [ ]:
with open("test.pickle", "wb") as fd:
    pickle.dump(svm, fd)

In [ ]:
len(svm_res)

In [ ]:
for pair in svm_res:
    print(df.loc[list(pair)])
    print()

In [ ]:
df.loc[[439989, 1803687]]

# Normalization stuff

In [ ]:
df["Fornavn"] = df.Navn.apply(lambda s: s.split()[0])

In [ ]:
import recordlinkage.standardise as rlstd

In [ ]:
subset = df[df.Navn.str.match(r"^(kar|car)", case=False)]

In [ ]:
subset["Fornavn"] = subset.Navn.str.translate(utils.trans).apply(lambda s: s.split()[0])

In [ ]:
subset["soundex"] = rlstd.phonetic(subset.Fornavn, method="soundex")

In [ ]:
subset["nysiis"] = rlstd.phonetic(subset.Fornavn, method="nysiis")

In [ ]:
subset["metaphone"] = rlstd.phonetic(subset.Fornavn, method="metaphone")

In [ ]:
n = set()
for s in subset.itertuples():
    n.add((s.Fornavn, s.soundex, s.nysiis, s.metaphone))

In [ ]:
subset.loc[:, ("Fornavn", "soundex")]

In [ ]:
subset["nysiis"] = rlstd.phonetic(subset.Fornavn, method="nysiis")

In [ ]:
subset["gb"] = subset.Navn.apply()

# Hmm

In [ ]:
df2 = df.set_index(["FT", "Kipnr", "Løbenr"])

In [ ]:
df2["Include"] = False

In [ ]:
for year, data in dm.items():
    for kip, løb in data:
        df2.loc[(year, kip, løb), "Include"] = True
for year, data in dn.items():
    for kip, løb in data:
        df2.loc[(year, kip, løb), "Include"] = True